In [1]:
#宿題３

def readNumber(line, index):
    number = 0
    flag = 0
    keta = 1
    while index < len(line) and (line[index].isdigit() or line[index] == '.'):
        if line[index] == '.':
            flag = 1
        else:
            number = number * 10 + int(line[index])
            if flag == 1:
                keta *= 0.1
        index += 1
    token = {'type': 'NUMBER', 'number': number * keta}
    return token, index


def readPlus(line, index):
    token = {'type': 'PLUS'}
    return token, index + 1

def readMinus(line, index):
    token = {'type': 'MINUS'}
    return token, index + 1

def readMul(line, index):
    token = {'type': 'MUL'}
    return token, index + 1

def readDivi(line, index):
    token = {'type': 'DIVI'}
    return token, index + 1

def readLeft(line, index):
    token = {'type': 'LEFT'}
    return token, index + 1

def readRight(line, index):
    token = {'type': 'RIGHT'}
    return token, index + 1


def tokenize(line):
    tokens = []
    index = 0
    while index < len(line):
        if line[index].isdigit():
            (token, index) = readNumber(line, index)
        elif line[index] == '+':
            (token, index) = readPlus(line, index)
        elif line[index] == '-':
            (token, index) = readMinus(line, index)
        elif line[index] == '*':
            (token, index) = readMul(line, index)
        elif line[index] == '/':
            (token, index) = readDivi(line, index)
        elif line[index] == '(':
            (token, index) = readLeft(line, index)
        elif line[index] == ')':
            (token, index) = readRight(line, index)
        else:
            print ('Invalid character found: ' + line[index])
            exit(1)
        tokens.append(token)
    return tokens


def evaluate1(tokens):
    answer = 0
    index = 1
    tokens.insert(len(tokens), {'type': 'EMPTY'}) # Insert a dummy ' ' token
    count = 0 # *と/が使われた回数をカウント
    while index < len(tokens):
        if tokens[index]['type'] == 'NUMBER':
            if tokens[index - 1]['type'] == 'MUL':
                tokens[index - 2]['number'] = tokens[index - 2]['number'] * tokens[index]['number']
                # *以降２つずつずらす
                for i in range(index-1, len(tokens)-3):
                    tokens[i] = tokens[i+2]
                count += 2
                for i in range(len(tokens)-count-1,len(tokens)-count+1):
                    tokens[i] = {'type': 'EMPTY'}# ずらした分' 'に変える
                index -=1
            elif tokens[index - 1]['type'] == 'DIVI':
                tokens[index - 2]['number'] = tokens[index - 2]['number'] / tokens[index]['number']
                 # /以降２つずつずらす
                for i in range(index-1, len(tokens)-3):
                    tokens[i] = tokens[i+2]
                count += 2
                for i in range(len(tokens)-count-1,len(tokens)-count+1):
                    tokens[i] = {'type': 'EMPTY'}# ずらした分' 'に変える
                index -= 1
            elif tokens[index - 1]['type'] == 'PLUS':
                index += 1
            elif tokens[index - 1]['type'] == 'MINUS':
                index += 1
        index += 1
    for i in range(len(tokens)-count-1,len(tokens)-1):
        tokens[i] = {'type': 'EMPTY'}# ずらした分' 'に変える
    return tokens


def evaluate2(tokens):
    answer = 0
    tokens.insert(0, {'type': 'PLUS'}) # Insert a dummy '+' token
    index = 1
    while index < len(tokens):
        if tokens[index]['type'] == 'NUMBER':
            if tokens[index - 1]['type'] == 'PLUS':
                answer += tokens[index]['number']
            elif tokens[index - 1]['type'] == 'MINUS':
                answer -= tokens[index]['number']
        index += 1
    return answer


def bracket(tokens): # 括弧を先に計算
    lefts = []
    left = 0
    count = 0
    reserve = 0
    right = 0
    index = 0
    brackets = []
    while index < len(tokens): # 左括弧の位置を配列に保存
        if tokens[index]['type'] == 'LEFT':
            lefts.insert(count,index)
            count += 1
            index += 1
        else:
            index += 1
    index = 1
    while index < len(tokens):# 右括弧の位置を取得し、その直前の左括弧の位置も取得
        if tokens[index]['type'] == 'RIGHT':
            right = index
            left = 0
            for i in range(len(lefts)):
                if left < lefts[i] and left < right :
                    left = lefts[i]
                    reserve = i
            lefts[reserve] = 0
            for i in range(left + 1 , right):# bracketsに括弧の中身を代入
                brackets.insert(i - left - 1, tokens[i])
            
            brackets = evaluate1(brackets)# 括弧の中身を計算
            tokens[left] = {'type':'NUMBER'}
            tokens[left]['number'] = evaluate2(brackets)
            
            for i in range(right + 1, len(tokens)):
                tokens[i-right+left] = tokens[i]
            for i in range(len(tokens)-right+left, len(tokens)):# 計算し終わった括弧を空にする
                tokens[i] = {'type':'EMPTY'}
                
            for i in range(len(brackets)): # bracketsの初期化
                brackets[i] = {'type':'EMPTY'}
            
            index += (left-right) + 1

        else:
            index += 1
            
    return tokens

def test(line, expectedAnswer):
    tokens = tokenize(line)
    tokens = bracket(tokens)
    tokens = evaluate1(tokens)
    actualAnswer = evaluate2(tokens)
    
    if abs(actualAnswer - expectedAnswer) < 1e-8:
        print ("PASS! (%s = %f)" % (line, expectedAnswer))
    else:
        print ("FAIL! (%s should be %f but was %f)" % (line, expectedAnswer, actualAnswer))


# Add more tests to this function :)
def runTest():
    print ("==== Test started! ====")
    test("1+2", 3)
    test("1.0+2.1-3", 0.1)
    test("2*5", 10)
    test("3*5.0+1.0", 16)
    test("6.0/2.0-6", -3)
    test("3.0+4*2-1/5", 10.8)
    test("(2+3)", 5)
    test("1+(2+3)", 6)
    test("1+(2.0+3)-4", 2)
    test("(2*3)+1", 7)
    test("1-(2*3)", -5)
    test("(2+4)/3", 2)
    test("2*3*4", 24)
    test("(-6*5)/3", -10)
    test("(-2.0*5+1)+2*3", -3)
    test("1+(2+(3+(4+5)))", 15)
    test("1+(2+3*4-(3*4))*4", 9)
    print ("==== Test finished! ====\n")

runTest()

==== Test started! ====
PASS! (1+2 = 3.000000)
PASS! (1.0+2.1-3 = 0.100000)
PASS! (2*5 = 10.000000)
PASS! (3*5.0+1.0 = 16.000000)
PASS! (6.0/2.0-6 = -3.000000)
PASS! (3.0+4*2-1/5 = 10.800000)
PASS! ((2+3) = 5.000000)
PASS! (1+(2+3) = 6.000000)
PASS! (1+(2.0+3)-4 = 2.000000)
PASS! ((2*3)+1 = 7.000000)
PASS! (1-(2*3) = -5.000000)
PASS! ((2+4)/3 = 2.000000)
PASS! (2*3*4 = 24.000000)
PASS! ((-6*5)/3 = -10.000000)
PASS! ((-2.0*5+1)+2*3 = -3.000000)
PASS! (1+(2+(3+(4+5))) = 15.000000)
PASS! (1+(2+3*4-(3*4))*4 = 9.000000)
==== Test finished! ====



In [ ]:
while True:
    print ('> ', )
    line = input()
    tokens = tokenize(line)
    bracket(tokens)
    tokens = evaluate1(tokens)
    answer = evaluate2(tokens)
    print ("answer = %f\n" % answer)